<h1>New York City Airbnb Open Data</h1>

<h4>Step 1.Download the dataset from Kaggle.<br><br>
    Step 2.Data Cleaning and Processing using Python.<br><br>
    Step 3.Load data to MySQL for Analysis.</h4>

In [6]:
#Import the Libraries
import pandas as pd
import kaggle
import re
from unidecode import unidecode
from sqlalchemy import create_engine
import pymysql

In [7]:
pip install pymysql

Note: you may need to restart the kernel to use updated packages.


<h2>Step 1.Download the dataset from Kaggle.</h2>


In [9]:
!pip install kaggle

In [10]:
#download dataset using kaggle agpi
!kaggle datasets download dgomonov/new-york-city-airbnb-open-data


Dataset URL: https://www.kaggle.com/datasets/dgomonov/new-york-city-airbnb-open-data
License(s): CC0-1.0
new-york-city-airbnb-open-data.zip: Skipping, found more recently modified local copy (use --force to force download)


In [11]:
#extract file from zip file
import zipfile
zip_ref = zipfile.ZipFile('new-york-city-airbnb-open-data.zip') 
# extract file to dir
zip_ref.extractall() 
# close file
zip_ref.close()

In [12]:
#Read the file to a dataframe
airbnb_df=pd.read_csv("/Users/pkv/Desktop/Seema/kaggle/python_sql/New York City Airbnb Open Data/AB_NYC_2019.csv")
airbnb_df.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,2539,Clean & quiet apt home by the park,2787,John,Brooklyn,Kensington,40.64749,-73.97237,Private room,149,1,9,2018-10-19,0.21,6,365
1,2595,Skylit Midtown Castle,2845,Jennifer,Manhattan,Midtown,40.75362,-73.98377,Entire home/apt,225,1,45,2019-05-21,0.38,2,355
2,3647,THE VILLAGE OF HARLEM....NEW YORK !,4632,Elisabeth,Manhattan,Harlem,40.80902,-73.94190,Private room,150,3,0,NaN,NaN,1,365
3,3831,Cozy Entire Floor of Brownstone,4869,LisaRoxanne,Brooklyn,Clinton Hill,40.68514,-73.95976,Entire home/apt,89,1,270,2019-07-05,4.64,1,194
4,5022,Entire Apt: Spacious Studio/Loft by central park,7192,Laura,Manhattan,East Harlem,40.79851,-73.94399,Entire home/apt,80,10,9,2018-11-19,0.10,1,0


In [13]:
airbnb_df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365'],
      dtype='object')

In [14]:
airbnb_df.shape


(48895, 16)

In [15]:
print("Records:",airbnb_df.shape[0]," & Columns: ",airbnb_df.shape[1])

Records: 48895  & Columns:  16


In [16]:
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

<h2>Step 2.Data cleaning</h2>

<h4>Checking for missing values in any columns of the dataframe</h4>

In [19]:
#Checking if any null values in any columns
airbnb_df.isnull().sum()

id                                    0
name                                 16
host_id                               0
host_name                            21
neighbourhood_group                   0
neighbourhood                         0
latitude                              0
longitude                             0
room_type                             0
price                                 0
minimum_nights                        0
number_of_reviews                     0
last_review                       10052
reviews_per_month                 10052
calculated_host_listings_count        0
availability_365                      0
dtype: int64

In [20]:
#Taking the percenatge of null values
round((airbnb_df.isnull().sum()/airbnb_df.shape[0])*100,2)

id                                 0.00
name                               0.03
host_id                            0.00
host_name                          0.04
neighbourhood_group                0.00
neighbourhood                      0.00
latitude                           0.00
longitude                          0.00
room_type                          0.00
price                              0.00
minimum_nights                     0.00
number_of_reviews                  0.00
last_review                       20.56
reviews_per_month                 20.56
calculated_host_listings_count     0.00
availability_365                   0.00
dtype: float64

<b>last_review & reviews_per_month </b> has  20.56% null value<br>
<b>name</b> has 0.03%
<b>host_name </b> has 0.04%

In [22]:
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48879 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48874 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

In [23]:
#Check for Duplicates in the Entire DataFrame
airbnb_df.duplicated().sum()

0

<h4> There are no fully duplicated rows in the dataset.</h4>

<h4>Impute name and host_name with "No Name Provided"</h4>

In [26]:
#replace nan with "No Name Provided" both for name & host_name column
airbnb_df.fillna({'name':"No Name Provided",'host_name':"No Name Provided"},inplace=True)

In [27]:
len(airbnb_df[airbnb_df['name']=="No Name Provided"])

16

In [28]:
len(airbnb_df[airbnb_df['host_name']=="No Name Provided"])

21

In [29]:
airbnb_df[['name','host_name']].isnull().sum()

name         0
host_name    0
dtype: int64

Output is 0 for both columns, it means there are no more missing values.

<h4>Impute reviews_per_month with 0 </h4>

In [32]:
airbnb_df['reviews_per_month'].fillna(0,inplace=True)

/var/folders/0n/jqw5d9gn2w9bfwy9mgh285r80000gn/T/ipykernel_67964/4235193524.py:1: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  airbnb_df['reviews_per_month'].fillna(0,inplace=True)


In [33]:
airbnb_df[['reviews_per_month']].isnull().sum()

reviews_per_month    0
dtype: int64

No nan in reviews_per_month column

<h4>Converting last_review  to date type and impute nan with 0</h4>

In [36]:
#convert last_review to date type
airbnb_df['last_review'] = pd.to_datetime(airbnb_df['last_review'], errors='coerce')
#replace nan with 0
airbnb_df['last_review'] = airbnb_df['last_review'].fillna(0)

In [37]:
airbnb_df['last_review'].isnull().sum()

0

No nan in last_review column

In [39]:
airbnb_df.isnull().sum()

id                                0
name                              0
host_id                           0
host_name                         0
neighbourhood_group               0
neighbourhood                     0
latitude                          0
longitude                         0
room_type                         0
price                             0
minimum_nights                    0
number_of_reviews                 0
last_review                       0
reviews_per_month                 0
calculated_host_listings_count    0
availability_365                  0
dtype: int64

<h4> No missing values in airbnb_df </h4>

In [41]:
airbnb_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48895 entries, 0 to 48894
Data columns (total 16 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   id                              48895 non-null  int64  
 1   name                            48895 non-null  object 
 2   host_id                         48895 non-null  int64  
 3   host_name                       48895 non-null  object 
 4   neighbourhood_group             48895 non-null  object 
 5   neighbourhood                   48895 non-null  object 
 6   latitude                        48895 non-null  float64
 7   longitude                       48895 non-null  float64
 8   room_type                       48895 non-null  object 
 9   price                           48895 non-null  int64  
 10  minimum_nights                  48895 non-null  int64  
 11  number_of_reviews               48895 non-null  int64  
 12  last_review                     

last_review is converted to datetime64

<h3>Standardizing the "Name" Column: Converting Non-English Characters to English</h3>

In [44]:

# Function to check if a string contains non-English (non-ASCII) characters
def has_non_english(text):
    return bool(re.search(r'[^\x00-\x7F]', str(text)))  # Matches any character outside ASCII range

# Apply function to count rows with non-English characters
non_english_rows = airbnb_df[airbnb_df['name'].apply(has_non_english)]

# Print the count of affected rows
print(f"Rows with non-English characters: {len(non_english_rows)}")

Rows with non-English characters: 1163


In [45]:
print("Name column having non english characters:\n",non_english_rows['name'].sort_values().head(1163))

Name column having non english characters:
 37224    "Cabin" —Private Queen Bedroom in Jungly Apart...
31402    "San-Paraíso" 80s curated Retro 3 Bedroom LES Pad
26630                                     #2,单间双人房,(共用卫生间)
10969    $199 SPECIAL VERY QUIET WVILLGMTPKGCHLS 65”TV/...
28923                                  (Room 102)法拉盛舒适轻奢套房
                               ...                        
36481                                     한성 韓城 Han B (2F)
36681                                     한성 韓城 Han C (2F)
25823     ️CENTRALLY LOCATED️- Great for Families + Groups
27139                              ｳｨﾘｱﾑｽﾞﾊﾞｰｸﾞのかわいいお部屋です2
47408         ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡
Name: name, Length: 1163, dtype: object


In [46]:
#unidecode function  converts any non-English characters to their closest English equivalents.
#name_translate column contains the English equivalent of the name with non-English characters converted.
airbnb_df['name_translate'] = airbnb_df['name'].apply(lambda x: unidecode(str(x)))

Checking if any non english character is present in name column

In [48]:
# Apply function to count rows with non-English characters
non_english_rows = airbnb_df[airbnb_df['name_translate'].apply(has_non_english)]

# Print the count of affected rows
print(f"Rows with non-English characters: {len(non_english_rows)}")

Rows with non-English characters: 0


In [49]:
airbnb_df[['name','name_translate']].sort_values(by='name',ascending=False).head(100)

,name,name_translate
47408,"ﾏﾝﾊｯﾀﾝ､駅から徒歩4分でどこに行くのにも便利な場所!女性の方希望,ｷﾚｲなお部屋｡","manhatutan,Yi karaTu Bu 4Fen dedokoniXing kuno..."
27139,ｳｨﾘｱﾑｽﾞﾊﾞｰｸﾞのかわいいお部屋です2,uiriamusu:ha:+ku:nokawaiioBu Wu desu2
25823,️CENTRALLY LOCATED️- Great for Families + Groups,CENTRALLY LOCATED- Great for Families + Groups
36681,한성 韓城 Han C (2F),hanseong Han Cheng Han C (2F)
36481,한성 韓城 Han B (2F),hanseong Han Cheng Han B (2F)
...,...,...
26297,汤姆公寓,Tang Mu Gong Yu
45355,步行9分钟到缅街中心的独立电梯房,Bu Xing 9Fen Zhong Dao Mian Jie Zhong Xin De D...
45359,"步行9分钟到法拉盛缅街中心的电梯房,提供免费矿泉水可乐",Bu Xing 9Fen Zhong Dao Fa La Sheng Mian Jie Zh...
45461,"步行9分钟到法拉盛缅街中心的电梯房,提供免费矿泉水可乐",Bu Xing 9Fen Zhong Dao Fa La Sheng Mian Jie Zh...


<h4>Non-English characters in the name column were converted to their equivalent English characters in the name_translate column./b><h4>

<h3>Checking 1-to-1 relationship between host_id and host_name </h3>

In [52]:
#Check if each host_id is associated with exactly one host_name:
host_id_mapping = airbnb_df.groupby('host_id')['host_name'].nunique()

#Check if each host_name is associated with exactly one host_id:
host_name_mapping = airbnb_df.groupby('host_name')['host_id'].nunique()

is_host_id_1_to_1 = host_id_mapping.max() == 1  # True if all host_ids map to 1 host_name
is_host_name_1_to_1 = host_name_mapping.max() == 1  # True if all host_names map to 1 host_id

if is_host_id_1_to_1 and is_host_name_1_to_1:
    print("host_id and host_name are 1-to-1 mapped.")
else:
    print("host_id and host_name are not 1-to-1 mapped.")
    if(is_host_id_1_to_1== False):
        print("host_id doesn't map to 1 host_name")
    if(is_host_name_1_to_1== False): 
        print("host_name doesn't map to 1 host_name")

host_id and host_name are not 1-to-1 mapped.
host_name doesn't map to 1 host_name


In [53]:
# Find the host_names with more than one unique host_id
non_unique_host_names = airbnb_df.groupby('host_name')['host_id'].nunique()
non_unique_host_names = non_unique_host_names[non_unique_host_names > 1]

# Show the host_names with multiple host_ids
print(len(non_unique_host_names ),"records have more than one unique host_id\n")
print(non_unique_host_names.sort_values(ascending=False))

3126 records have more than one unique host_id

host_name
Michael    335
David      318
Alex       226
Daniel     208
Sarah      199
          ... 
Mio          2
Dipti        2
Dirk         2
Dj           2
소정           2
Name: host_id, Length: 3126, dtype: int64


In [54]:
print("There are ",airbnb_df['host_name'].nunique(),"unique host_name")

There are  11453 unique host_name


In [55]:
airbnb_df['host_name'].describe()

count       48895
unique      11453
top       Michael
freq          417
Name: host_name, dtype: object

In [56]:
# Get the counts of all host_name values
host_name_counts = airbnb_df['host_name'].value_counts()

# Print the counts for each host_name
#print(host_name_counts)

# To get the total number of host_name entries
total_host_names = host_name_counts.sum()
print(f"Total number of host_name entries: {total_host_names}")

Total number of host_name entries: 48895


In [57]:
print("the total count of missing (NaN) values in the host_name column ",airbnb_df['host_name'].isnull().sum())

the total count of missing (NaN) values in the host_name column  0


<h3>checking all the columns</h3>

In [59]:
airbnb_df.columns

Index(['id', 'name', 'host_id', 'host_name', 'neighbourhood_group',
       'neighbourhood', 'latitude', 'longitude', 'room_type', 'price',
       'minimum_nights', 'number_of_reviews', 'last_review',
       'reviews_per_month', 'calculated_host_listings_count',
       'availability_365', 'name_translate'],
      dtype='object')

In [60]:
airbnb_df['id'].nunique()

48895

In [61]:
airbnb_df['name_translate'].nunique()

47900

In [62]:
airbnb_df['host_id'].nunique()

37457

In [63]:
airbnb_df['host_name'].nunique()

11453

In [64]:
print(airbnb_df['neighbourhood_group'].nunique())
print(airbnb_df['neighbourhood_group'].unique())

5
['Brooklyn' 'Manhattan' 'Queens' 'Staten Island' 'Bronx']


In [65]:
print(airbnb_df['neighbourhood'].nunique())

221


In [66]:
airbnb_df['latitude'].nunique()

19048

In [67]:
airbnb_df['longitude'].nunique()

14718

In [68]:
print(airbnb_df['room_type'].nunique())
print(airbnb_df['room_type'].unique())

3
['Private room' 'Entire home/apt' 'Shared room']


In [69]:
airbnb_df['number_of_reviews'].nunique()

394

In [70]:
airbnb_df['last_review'].nunique()

1765

In [71]:
airbnb_df['reviews_per_month'].nunique()

938

In [72]:
airbnb_df['calculated_host_listings_count'].nunique()

47

In [122]:
airbnb_df['availability_365'].nunique()

366

<h3>Cleaning of the data is done</h3>

<h2>  Step 3.Load data to MySQL for Analysis.</h2>

In [126]:
#. Establish Connection to MySQL
db_host = "127.0.0.1:3306" 

db_name = "airbnb"
table_name = "airbnb_table"
db_user = "root"
try:
    # Create an engine without user and password
    engine = create_engine(f"mysql+pymysql://{db_user}@{db_host}/{db_name}")
# Test the connection
    with engine.connect() as connection:
        print("✅ Successfully connected to MySQL!")


    #Copy DataFrame to MySQL
    airbnb_df.to_sql(name=table_name, con=engine, if_exists="append", index=False)
    print(f"✅ Data successfully copied to table '{table_name}' in database '{db_name}'!")
except Exception as e:
    print(f"❌ Error: {e}")

finally:
    engine.dispose()
    print("🔌 Connection closed.")

✅ Successfully connected to MySQL!
✅ Data successfully copied to table 'airbnb_table' in database 'airbnb'!
🔌 Connection closed.


<h3>In this notebook:

* Successfully downloaded the dataset from Kaggle.

* Cleaned and processed the data:

    - Handled missing values

    - Converted non-English characters to English

* Loaded the data into MySQL for analysis using Python<h3>